In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.parse import quote
from time import sleep
import re

In [ ]:
bayern_hist = pd.read_excel(r'C:\Users\mariu\Documents\Project Local Elections\Bayern_historisch.xlsx')

In [ ]:
bayern_hist_2 = bayern_hist.drop(['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], 1)
bayern_hist_2.head()

In [ ]:
bayern_hist_2['Doctor'] = bayern_hist_2['Name'].str.contains('Dr.', regex=False)
bayern_hist_2.iloc[:,1:] = bayern_hist_2.iloc[:,1:].replace({True:1,False:0})

In [ ]:
def pre_name(data):
    
    """ Here, we are going to separate
        first and last name as well as any title """

    new = data['Name'].str.split(",", expand = True)
    data["Nachname"]= new[0]
    data["Vorname"]= new[1]
    data.Nachname = data.Nachname.str.replace('Dr.', ' ')

pre_name(bayern_hist_2)

def combine_name(data):

    """ Combine separate columns to get googleable name """

    return data.assign(Name_compl=data.agg('{0[Vorname]} {0[Nachname]}'.format, axis=1))

# Split dataframe to make waiting times bearable
bayern_hist_3 = combine_name(bayern_hist_2)
bayern_hist_3['Jahr'] = bayern_hist_3['Jahr'].apply(str)
bayern_hist_4 = pd.DataFrame(data=bayern_hist_3[0:4000])
bayern_hist_5 = pd.DataFrame(data=bayern_hist_3[4000:8000].reset_index(drop = True))
bayern_hist_6 = pd.DataFrame(data=bayern_hist_3[8000:12000].reset_index(drop = True))
bayern_hist_7 = pd.DataFrame(data=bayern_hist_3[12000:16000].reset_index(drop = True))
bayern_hist_8 = pd.DataFrame(data=bayern_hist_3[16000:].reset_index(drop = True))

In [ ]:
# To avoid being blocked by Ecosia (Electronical ID)
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    }

def beautiful_soup(names, year, city):
    
    """ Script to scrape ecosia on a large scale. 
        As a goal, we retrieve the number of search results."""

    results_next = []
    new_core = []

    for i in range(len(names)):

        page_link = 'https://www.ecosia.org/search?q=' + names[i] + ' ' + year[i] + ' ' + city[i] # Search query

        sleep(2)

        page_response = requests.get(page_link, headers = headers) # Results page

        page_content = BeautifulSoup(page_response.content, "html.parser")
        text = page_content.get_text() # Pure text of website

        core = re.findall(r"\n\n\n\n\n\n\n\n\n\n\n(.*)", text) # Regex for search result structure

        new_core.append(core[27])

        results_next.append([names[i], year[i], new_core[i]])

    data_google = pd.DataFrame(data=results_next, columns = ['Name_total', 'year', 'google']) # Put in Dataframe

    return data_google

In [ ]:
bayern_hist_a = beautiful_soup(bayern_hist_4['Name_compl'], bayern_hist_4['Jahr'], bayern_hist_4['Stadt'])"

In [ ]:
bayern_hist_b = beautiful_soup(bayern_hist_5['Name_compl'], bayern_hist_5['Jahr'], bayern_hist_5['Stadt'])

In [ ]:
bayern_hist_c = beautiful_soup(bayern_hist_6['Name_compl'], bayern_hist_6['Jahr'], bayern_hist_6['Stadt'])

In [ ]:
bayern_hist_d = beautiful_soup(bayern_hist_7['Name_compl'], bayern_hist_7['Jahr'], bayern_hist_7['Stadt'])

In [ ]:
bayern_hist_e = beautiful_soup(bayern_hist_8['Name_compl'], bayern_hist_8['Jahr'], bayern_hist_8['Stadt'])

In [ ]:
bayern_hist_10 = bayern_hist_a.append(bayern_hist_b, ignore_index = True).append(bayern_hist_c, ignore_index = True).append(bayern_hist_d, ignore_index = True).append(bayern_hist_e, ignore_index = True)
frames = [bayern_hist_3, bayern_hist_10]
bayern_hist_11 = pd.concat(frames, ignore_index=True, axis=1)
bayern_hist_11.to_excel(r'C:\Users\mariu\Documents\Project Local Elections\bayern_historisch_search.xlsx')